# A toy trading system [Test]

I wanted to work on the node class, but that's hard without having an application/demo that is meaty enough to
work with.

So, here's a toy front-office position management, pricing, and risk system:

In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, DynamoDbDriver, ddb, _tr, Timestamp, Context, _DBO
from mand.core import PrintMonitor, SummaryMonitor, ProfileMonitor
from mand.lib.refdata import RefData, RefDataUpdateEvent
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayTable
import datetime

db = ObjectDb()

## Books and Portfolios [BA]

These are just workflow entities that support a net present value function. As long as items in books are valuable
(in the sense of implementing NPV) then everythin just works.

In [2]:
class TradingBook(Workbook):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            #print i, i.name(), i.NPV(), q, i.NPV()*q
            ret += i.NPV() * q
        return ret

class TradingPortfolio(Portfolio):
    @node
    def NPV(self):
        return sum( [c.NPV() for c in self.children()] )
    
_tr.add(TradingBook)
_tr.add(TradingPortfolio)

## Building a tree of bank-side accounts [User]

Here, we build a tree of 100 books, grouped under 10 sub-portfolios, rooted in one top-level portfolio.

In a real bank, we might have 100,000 books, grouped 10 levels deep. in multiple different trees. Plus millions 
of customer books grouped in various portfolio trees.

In [3]:
def makeTree(name, depth):
    if depth:
        subs = [ makeTree(name+str(i), depth-1) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        return p
    else:
        ret = TradingBook(name).write()
        return ret

with db:
    pAll = makeTree('B', 2)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 100
# children: 10


## Checking execution counts [Test]

We have 11 portfolios, 100 books. We should get node calculation counts that reflect this...

In [4]:
with ProfileMonitor(mode='sum'):
    db2 = db.copy()
    p = db2._get(pAll.meta.path())
    print len(p.books())

100


Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:books|11|424,331|104|38,575|GetValue
|Portfolio:books|11|424,226|475|38,566|GetValue/Calc
|Portfolio:children|11|224,750|117|20,431|GetValue
|Portfolio:children|11|224,633|19,616|20,421|GetValue/Calc
|PortfolioUpdateEvent:children|11|176,675|964|16,061|GetValue
|TradingBook|100|158,687|158,687|1,586|Db.Get
|PortfolioUpdateEvent|11|24,425|24,425|2,220|Db.Get
|TradingPortfolio|10|17,023|17,023|1,702|Db.Get
|RootClock|1|4,137|4,137|4,137|Db.Get
|Clock:cutoffs|22|1,980|68|90|GetValue
|Clock:cutoffs|1|1,911|21|1,911|GetValue/Calc
|Clock:parent|1|1,888|8|1,888|GetValue
|Clock:parent|1|1,880|1,760|1,880|GetValue/Calc
|Portfolio:clock|22|1,875|160|85|GetValue
|Portfolio:clock|11|1,715|148|155|GetValue/Calc
|CosmicAll|1|1,680|1,680|1,680|Db.Get
|Clock|1|1,567|1,567|1,567|Db.Get
|RootClock:cutoffs|3|72|13|24|GetValue
|RootClock:cutoffs|1|59|21|59|GetValue/Calc
|Event:amends|11|58|58|5|GetValue
|Entity:clock|2|49|40|24|GetValue
|RootClock:cosmicAll|1|26|18|26|GetValue
|CosmicAll:dbState|1|10|6|10|GetValue
|Entity:clock|1|8|8|8|GetValue/Calc
|RootClock:cosmicAll|1|8|8|8|GetValue/Calc
|CosmicAll:dbState|1|3|3|3|GetValue/Calc

## A few books [Test]

We also create two customers' books.

Pretend *p1* is one of our trading desks, *p2* is another. 

In [5]:
with db:
    bExt = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[3]

b1 = p1.children()[4]
b2 = p2.children()[7]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()



Customer1
B04
B37


## Tracking Market Data [DBA]

For now, just save incoming market observations as reference data. 

In reality, we'd have billions of observations per day on millions of data sources.

In [6]:
class MarketDataSource(ExternalRefData):
    @dataField
    def last(self):
        return None
    
_tr.add(MarketDataSource)

## Making some data sources [Test]

Two pretend data sources: IBM and Google last trade prices.

In [7]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

print s1_ibm.last()
print s1_goog.last()

175.61
852.12


## Market Interface [DBA]

How user code accesses market data. 

In reality, market interfaces would be choosing amongst raw data sources,
providing bootstrapped curves, vol surfaces, etc.

In [8]:
class MarketInterface(Entity):
    
    @node
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node
    def spot(self):
        return self.source().last()
                           
    
_tr.add(MarketInterface)


## Making some market interfaces [Test]

In [9]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

print ibm.spot(), type(ibm.spot())
print goog.spot(), type(goog.spot())

175.61 <class 'decimal.Decimal'>
852.12 <class 'decimal.Decimal'>


## Instruments [BA]

Things we can own/have contracted. Instruments have an NPV.

We implement two classes of instrument:
* ForwardCashflow: money that will arrive at some future time
* Equity: a share of common stock in a company
    
Both implementations are toys.

In [10]:
class Instrument(WorkItem):
    """A thing that can be owned, an asset, or legal obligation"""
    
class ForwardCashflow(Instrument):
    
    @node(stored=True)
    def currency(self):
        return 'USD'
    
    @node(stored=True)
    def settlementDatetime(self):
        d = datetime.datetime.utcnow() + datetime.timedelta(2)
        return d
    
    @node
    def NPV(self):
        # XXX - would really get the currency discount curve here, and discount according to 
        # current time/settlement time
        # XXX - and do a conversion to our native currency
        return 1
    
    @node
    def name(self):
        return 'Cash %s/%s' % (self.currency(), self.settlementDatetime())
    
class Equity(Instrument):
    
    @node(stored=True)
    def assetName(self):
        return 'IBM.Eq.1'
    
    @node
    def NPV(self):
        return self.refdata().spot()
    
    @node
    def refdata(self):
        return self.getObj(_tr.MarketInterface, self.assetName().split('.')[0])

    @node
    def name(self):
        return 'Stock: %s' % self.assetName()
    
_tr.add(ForwardCashflow)
_tr.add(Equity)

## TradeOpenEvent [BA]

An event/observation that represents buying or selling something.

Someone gets *quantity* of *item*, and pays *unitPrice* of *premium* for each *item*.

Note that *premium* is typically a *ForwardCashflow*, so the money settles T+2. From a risk/PnL point
of view, we get the asset immediately. This is a gross simplification.

In [11]:
class TradeOpenEvent(WorkItemOpenEvent):
    @node(stored=True)
    def action(self):
        return 'Buy'
    @node(stored=True)
    def quantity(self):
        return 1.
    @node(stored=True)
    def premium(self):
        return None
    @node(stored=True)
    def unitPrice(self):
        return 0.
    
    def _items(self):
        bs = 1 if self.action() == 'Buy' else -1
        pq = -bs * self.unitPrice() * self.quantity()
        return [ [ self.ticket(), self.item(),    bs*self.quantity(), self.book1(), self.book2() ],
                 [ self.ticket(), self.premium(), pq,                 self.book1(), self.book2() ]
               ] 

_tr.add(TradeOpenEvent)

## Let's book some trades [Test]

In [12]:
with db:
    cf1 = ForwardCashflow()
    ins1 = Equity()
    ins2 = Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.68,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                        ).write()
    
    ts6 = Timestamp()
    

# Reporting [User]

Reporting is the whole point of the Abstract Nonsense Db.

We ignore all the standard report infrastructure goop (aggregation, GUIs, pivot-tables, drill-down, etc) and
just focus on extracting the underlying data.

For the following examples, we mostly care about NPV (the present values of what we own,) 
PnL (how much money we made or lost,) and risk (what happens to our NPV if underlying conditions change.)

Note that even in the trivial case of our three trades, one amendment, and five market data updates, trying to write
code to attribute profit, or even trying to figure it by hand, would require assumptions and be error-prone. 

In general, the less a report knows about the underlying data, the better it is.

## PnL: The most basic report

How much money did we (i.e. our portfolio) gain or lose between two times?

For example, if time1 was yesterday's close, and time2 is today's close, this report tells us how much we made or lost
today. Will we be drinking champagne or straight vodka this evening?

Note: it's worth understanding how the cash entries are working in this example. The reason our open and closed out
positions are pricing rationally is because we are modelling cash approximately correct, not because the instruments
are carrying around a notion of price-paid!

In [13]:
def reportPnL(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    with Context({clock.cutoffs: ts2}):
        npv2 = valuable.NPV()
    print 'PnL:', npv2-npv1

### Example: top-of-the-house intraday PnL

Hmm, one of our desks bought 100 shares of IBM, and the last trade price is already down 1 cent. The bank is down 
100 times $.01 or one dollar:

In [14]:
# 100 shares bought at $175.65
# current price is $175.64:
reportPnL(pAll, ts1, ts3)

PnL: -1.00


### Example: top-of-the-house daily PnL

By end of day, the bank as a whole has closed out the IBM trade for a profit, and bought some GOOG. Closing refdata
prices have been entered. We should have a total profit for the day of $23:

In [15]:
# 100 * ($175.85 - $175.65) =  $20
# 300 * ($852.13 - $852.12) =  $ 3
#                              ----
#                              $23
reportPnL(pAll, ts1, eod)

PnL: 23.00


### Example: trading desk daily PnL

Desk *p1* bought IBM at $175.65

It still owns IBM, which closed at $175.68

So, it should be up $3:

In [16]:
# 100 * ($175.68 - $175.65) =   $3
reportPnL(p1, ts1, eod)

PnL: 3.00


# Let's see what is going on when we run a report [Core]

In [17]:
with ProfileMonitor(mode='sum'):
    db3 = db.copy()
    p = db3._get(pAll.meta.path())
    reportPnL(p, ts1, ts6)

PnL: 20.00


Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|TradingPortfolio:NPV|22|1,302,152|230|59,188|GetValue
|TradingPortfolio:NPV|22|1,301,921|2,797|59,178|GetValue/Calc
||2|664,976|40|332,488|Context
|TradingBook:NPV|200|411,641|1,662|2,058|GetValue
|TradingBook:NPV|200|409,979|2,725|2,049|GetValue/Calc
|Workbook:items|200|376,449|1,686|1,882|GetValue
|Workbook:items|200|374,762|339,107|1,873|GetValue/Calc
|Portfolio:children|22|250,266|208|11,375|GetValue
|Portfolio:children|22|250,057|38,796|11,366|GetValue/Calc
|PortfolioUpdateEvent:children|22|180,865|1,149|8,221|GetValue
|TradingBook|102|167,066|167,066|1,637|Db.Get
|Equity:NPV|3|30,775|24|10,258|GetValue
|Equity:NPV|2|30,750|53|15,375|GetValue/Calc
|MarketInterface:spot|2|27,426|17|13,713|GetValue
|MarketInterface:spot|2|27,409|65|13,704|GetValue/Calc
|PortfolioUpdateEvent|11|24,378|24,378|2,216|Db.Get
|ExternalRefData:state|2|23,916|16|11,958|GetValue
|ExternalRefData:state|2|23,899|37|11,949|GetValue/Calc
|RefData:state|2|23,862|24|11,931|GetValue
|RefData:state|2|23,837|4,044|11,918|GetValue/Calc
|RefDataUpdateEvent|7|16,397|16,397|2,342|Db.Get
|TradingPortfolio|10|15,849|15,849|1,584|Db.Get
|Clock:cutoffs|448|10,095|1,398|22|GetValue
|TradeOpenEvent|4|9,122|9,122|2,280|Db.Get
|Clock:cutoffs|5|8,697|105|1,739|GetValue/Calc
|Clock:parent|5|8,576|46|1,715|GetValue
|Clock:parent|5|8,530|8,364|1,706|GetValue/Calc
|Workbook:clock|400|8,206|3,694|20|GetValue
|_WorkItemEvent:ticket|6|4,917|54|819|GetValue
|Clock|3|4,900|4,900|1,633|Db.Get
|WorkTicket|3|4,862|4,862|1,620|Db.Get
|Workbook:clock|200|4,512|2,843|22|GetValue/Calc
|RootClock|1|3,946|3,946|3,946|Db.Get
|_WorkItemEvent:item|3|3,519|39|1,173|GetValue
|Equity|2|3,479|3,479|1,739|Db.Get
|MarketInterface:source|2|3,427|20|1,713|GetValue
|MarketInterface:source|2|3,407|58|1,703|GetValue/Calc
|MarketDataSource|2|3,325|3,325|1,662|Db.Get
|Equity:refdata|2|3,270|24|1,635|GetValue
|Equity:refdata|2|3,246|62|1,623|GetValue/Calc
|_WorkItemEvent:book2|6|3,240|40|540|GetValue
|MarketInterface|2|3,174|3,174|1,587|Db.Get
|Portfolio:clock|44|2,333|373|53|GetValue
|Portfolio:clock|22|1,959|294|89|GetValue/Calc
|CosmicAll|1|1,819|1,819|1,819|Db.Get
|TradeOpenEvent:premium|3|1,685|29|561|GetValue
|ForwardCashflow|1|1,656|1,656|1,656|Db.Get
|RefData:clock|4|1,626|26|406|GetValue
|RefData:clock|2|1,599|33|799|GetValue/Calc
|Event:amends|32|186|186|5|GetValue
|Entity:clock|10|138|85|13|GetValue
|Entity:clock|5|52|52|10|GetValue/Calc
|RootClock:cutoffs|15|42|42|2|GetValue
|RefDataUpdateEvent:data|7|37|37|5|GetValue
|_WorkItemEvent:book1|6|33|33|5|GetValue
|ForwardCashflow:NPV|2|28|24|14|GetValue
|TradeOpenEvent:quantity|6|27|27|4|GetValue
|MarketInterface:sourceName|2|23|17|11|GetValue
|TradeOpenEvent:unitPrice|3|20|20|6|GetValue
|TradeOpenEvent:action|3|18|18|6|GetValue
|Equity:assetName|2|10|10|5|GetValue
|MarketInterface:sourceName|2|5|5|2|GetValue/Calc
|ForwardCashflow:NPV|1|4|4|4|GetValue/Calc